In [ ]:
import json
import numpy as np
from decimal import Decimal

## Open File

In [ ]:
# Скачивание датасета:
# http://imopse.ii.pwr.wroc.pl/download.html

In [ ]:
file_name = "200_20_145_15"
with open(f"data_imopse_raw/{file_name}.def") as file:
    data = file.read()
    data = data.replace(" ", "")
    data = data.split("\n")

## Общая информация

In [ ]:
sections = [i for i, value in enumerate(data) if value == "="*116]
sections

In [ ]:
section_general = sections[1]

general_data = {
    "n_tasks": int(data[section_general+2].split(":")[1]),
    "n_workers": int(data[section_general+3].split(":")[1]),
    "n_precedence": int(data[section_general+4].split(":")[1]),
    "n_skill_types": int(data[section_general+5].split(":")[1]),
}
general_data

## Сотрудники

In [ ]:
section_resources = sections[2]

In [ ]:
def dict_to_vector_skill(d):
    vec = [0 for i in range(general_data["n_skill_types"])]
    for skill, level in d.items():
        vec[int(skill.replace("Q", ""))] = level + 1
    return vec

dict_to_vector_skill({'Q4': 2, 'Q8': 1, 'Q7': 0, 'Q2': 1, 'Q6': 2, 'Q13': 1})

In [ ]:
resources = data[section_resources+2: section_resources+2+general_data["n_workers"]]

worker_pays = {}
worker_skill_vectors = {}

for i in resources:
    id_, _, _, pay, _, *skills = i.split("\t")
    id_ = int(id_) - 1
    skills = [i for i in skills if i != ""]
    
    worker_pays[int(id_)] = int(Decimal(pay)*100)
    worker_skill_vectors[int(id_)] = dict_to_vector_skill({
        i.split(":")[0]: int(i.split(":")[1])
        for i in skills
    })

## Задачи

In [ ]:
tasks = [i.split("\t") for i in data[sections[3]+2:sections[3]+2+general_data["n_tasks"]]]

In [ ]:
tast_durations = {}
task_skill_vectors = {}
task_depends_from = {}

for i in tasks:
    id_, _, _, duration, skill, _, *depend = i
    depend = [int(i) for i in depend if i != ""]
    
    tast_durations[int(id_)] = int(duration)
    task_skill_vectors[int(id_)] = dict_to_vector_skill(
        {skill.split(":")[0]: int(skill.split(":")[1])}
    )
    task_depends_from[int(id_)] = depend
    
task_depends_from

In [ ]:
N_TASKS = general_data["n_tasks"]
N_SKILLS = general_data["n_skill_types"]

# Добавляем фиктивные задачи
# "Начало всего проекта" и "конец всего проекта"
tast_durations[0] = 1
tast_durations[N_TASKS+1] = 1

task_skill_vectors[0] = [0 for i in range(N_SKILLS)]
task_skill_vectors[N_TASKS+1] = [0 for i in range(N_SKILLS)]

task_depends_from = {
    key: value or [0]
    for key, value in task_depends_from.items()
}
task_depends_from[N_TASKS+1] = [i for i in task_depends_from.keys()]
task_depends_from[0] = []

In [ ]:
with open(f"imopse_parsed/{file_name}.json", "w") as save_file:
    result = {
        "info": general_data,
        "worker_pays": worker_pays,
        "worker_skill_vectors": worker_skill_vectors,
        
        "tast_durations": tast_durations,
        "task_skill_vectors": task_skill_vectors,
        "task_depends_from": task_depends_from,
    }
    json.dump(result, save_file)